In [47]:
import distillation.amundson_1958 as am

model = am.Model(
    components=['n-Butane', 'n-Pentane', 'n-Octane'],
    F=1000., # kmol/h
    P = 101325.*2, # ~ 2bar
    z_feed=[0.2, 0.5, 0.3], 
    RR=1.,
    D=400.,
    N=30,
    feed_stage=15,
)

In [48]:
model.add_parameters(verbose=True)

Setting DePriester parameters for n-Butane:
             a_T1 [deg Rankine^2]: -1280557.0
             a_T2 [deg Rankine]: 0.0
             a_T6 [dimensionless]: 7.94986
             a_p1 [dimensionless]: -0.96455
             a_p2 [psia^2]: 0.0
             a_p3 [psia]: 0.0
     K value at 300 K, 1 bar=  2.661186999988651
Setting DePriester parameters for n-Pentane:
             a_T1 [deg Rankine^2]: -1524891.0
             a_T2 [deg Rankine]: 0.0
             a_T6 [dimensionless]: 7.33129
             a_p1 [dimensionless]: -0.89143
             a_p2 [psia^2]: 0.0
             a_p3 [psia]: 0.0
     K value at 300 K, 1 bar=  0.7541583496653457
Setting DePriester parameters for n-Octane:
             a_T1 [deg Rankine^2]: 0.0
             a_T2 [deg Rankine]: -7646.81641
             a_T6 [dimensionless]: 12.48457
             a_p1 [dimensionless]: -0.73152
             a_p2 [psia^2]: 0.0
             a_p3 [psia]: 0.0
     K value at 300 K, 1 bar=  0.026447701262845617
Setting heat capac

In [49]:
model.calculate_T_feed()
print(model.T_feed)

334.27687743168434


In [50]:
model.initialize_flow_rates()
print(model.L)
print(model.V)

[ 400.  400.  400.  400.  400.  400.  400.  400.  400.  400.  400.  400.
  400.  400.  400. 1400. 1400. 1400. 1400. 1400. 1400. 1400. 1400. 1400.
 1400. 1400. 1400. 1400. 1400. 1400.  600.]
[  0. 800. 800. 800. 800. 800. 800. 800. 800. 800. 800. 800. 800. 800.
 800. 800. 800. 800. 800. 800. 800. 800. 800. 800. 800. 800. 800. 800.
 800. 800. 800.]


In [51]:
model.update_K_values()
import pprint
pprint.pprint(model.K)

{'n-Butane': array([3.16431133, 3.16431133, 3.16431133, 3.16431133, 3.16431133,
       3.16431133, 3.16431133, 3.16431133, 3.16431133, 3.16431133,
       3.16431133, 3.16431133, 3.16431133, 3.16431133, 3.16431133,
       3.16431133, 3.16431133, 3.16431133, 3.16431133, 3.16431133,
       3.16431133, 3.16431133, 3.16431133, 3.16431133, 3.16431133,
       3.16431133, 3.16431133, 3.16431133, 3.16431133, 3.16431133,
       3.16431133]),
 'n-Octane': array([0.06739293, 0.06739293, 0.06739293, 0.06739293, 0.06739293,
       0.06739293, 0.06739293, 0.06739293, 0.06739293, 0.06739293,
       0.06739293, 0.06739293, 0.06739293, 0.06739293, 0.06739293,
       0.06739293, 0.06739293, 0.06739293, 0.06739293, 0.06739293,
       0.06739293, 0.06739293, 0.06739293, 0.06739293, 0.06739293,
       0.06739293, 0.06739293, 0.06739293, 0.06739293, 0.06739293,
       0.06739293]),
 'n-Pentane': array([1.1114709, 1.1114709, 1.1114709, 1.1114709, 1.1114709, 1.1114709,
       1.1114709, 1.1114709, 1.1114709, 1

In [52]:
for i in model.components:
    model.solve_component_mass_bal(i)

for j in model.stages:
    for component in model.components:
        print(j, component, model.l[component][j])

0 n-Butane 199.99490523700513
0 n-Pentane 146.74061229497946
0 n-Octane 6.067598691939356e-12
1 n-Butane 63.20329584418709
1 n-Pentane 132.02380020603005
1 n-Octane 9.003316871673067e-11
2 n-Butane 41.588543840351925
2 n-Pentane 125.40337853641289
2 n-Octane 7.129885351025584e-10
3 n-Butane 38.173147923445455
3 n-Pentane 122.42515316949068
3 n-Octane 5.334804219725672e-09
4 n-Butane 37.633473479475356
4 n-Pentane 121.0853855868781
4 n-Octane 3.962486956832107e-08
5 n-Butane 37.5481982983367
5 n-Pentane 120.48268533613299
5 n-Octane 2.940289094184788e-07
6 n-Butane 37.534723773834976
6 n-Pentane 120.21155801751262
6 n-Octane 2.1814973302021034e-06
7 n-Butane 37.53259463370155
7 n-Pentane 120.08959021874134
7 n-Octane 1.6184958287637132e-05
8 n-Butane 37.53225820344703
8 n-Pentane 120.03472247611471
8 n-Octane 0.0001200791038145823
9 n-Butane 37.53220504334057
9 n-Pentane 120.01003998441388
9 n-Octane 0.0008908880850788226
10 n-Butane 37.53219664339176
10 n-Pentane 119.99893645860077
10 

In [53]:
print(model.T)
model.update_T_values()
print(model.T)

[334.27687743 334.27687743 334.27687743 334.27687743 334.27687743
 334.27687743 334.27687743 334.27687743 334.27687743 334.27687743
 334.27687743 334.27687743 334.27687743 334.27687743 334.27687743
 334.27687743 334.27687743 334.27687743 334.27687743 334.27687743
 334.27687743 334.27687743 334.27687743 334.27687743 334.27687743
 334.27687743 334.27687743 334.27687743 334.27687743 334.27687743
 334.27687743]
[301.82534722 311.63495344 315.17841269 315.74902551 315.77870427
 315.75373571 315.73634827 315.72754918 315.72345672 315.72175236
 315.72217974 315.73125911 315.80122539 316.3183096  319.98978303
 340.95411326 348.27903189 353.34003466 356.54216728 358.46060122
 359.57721206 360.22286821 360.60502455 360.84985925 361.03629974
 361.21975817 361.4491169  361.78257185 362.3384419  364.19160078
 384.4690221 ]


In [54]:
print(model.T_is_converged())

False


In [ ]:
iter = -1
while not model.T_is_converged():
    iter += 1
    model.update_K_values()
    for i in model.components:
        model.solve_component_mass_bal(i)
    model.update_T_values()
    print(iter, model.T)

0 [307.44145427 318.3288886  322.31222428 323.66824878 324.22168722
 324.51435712 324.69984698 324.82810354 324.92008308 324.98710429
 325.03632465 325.07270554 325.10253579 325.18546063 326.22914368
 336.08871564 339.20562437 341.06847019 342.62068343 344.28568132
 346.23377782 348.5429233  351.267435   354.4652354  358.20971875
 362.59737413 367.7594954  373.92525307 381.94719602 396.00914458
 414.72647797]
1 [307.02406383 317.69326419 321.37710376 322.31565008 322.49285368
 322.48192422 322.43199597 322.38104259 322.33796688 322.30382423
 322.2776695  322.25926645 322.26049616 322.41998583 324.14440679
 337.13684789 339.46040892 339.35947015 338.40161352 337.37637423
 336.57209573 336.07089053 335.90064483 336.10780223 336.81011714
 338.27787741 341.13556665 347.01624474 360.50201924 386.81658056
 410.97831365]
2 [301.86154363 312.39075053 316.9391754  318.26733398 318.61673393
 318.7169855  318.75731916 318.78311047 318.80435893 318.82286273
 318.83889306 318.85377304 318.88312166 

In [ ]:
print(model.V)
model.solve_energy_balances()
print(model.V)
print(model.flow_rates_converged())

In [ ]:
inner_iter = -1
outer_iter = -1
while not model.flow_rates_converged():
    outer_iter += 1
    for i in model.components:
        model.solve_component_mass_bal(i)
    model.update_T_values()
    while not model.T_is_converged():
        inner_iter += 1
        model.update_K_values()
        for i in model.components:
            model.solve_component_mass_bal(i)
        model.update_T_values()
    model.solve_energy_balances()
    print(outer_iter, inner_iter, model.V)
    

In [ ]:
print(model.L)
print(model.V)

In [ ]:
x = {}
y = {}
for i in model.components:
    x[i] = model.l[i][:]/model.L[:]
    y[i] = model.K[i]*x[i]

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(1)
ax = fig.add_subplot(111)
ax.plot(model.stages, model.T, 'o')
ax.set_xlabel('Stage Number')
ax.set_ylabel('Temperature [K]')

# plot liquid-phase mole fractions
fig2 = plt.figure(2)
ax2 = fig2.add_subplot(111)
# calculate mole fractions
for i in model.components:
    ax2.plot(model.stages, x[i], label=i)
ax2.set_ylabel('Liquid phase mole fraction')
ax2.set_xlabel('Stage Number')
ax2.legend()


fig3 = plt.figure(3)
ax3 = fig3.add_subplot(111)
# calculate mole fractions
for i in model.components:
    ax3.plot(model.stages, y[i], label=i)
ax3.set_ylabel('Vapor phase mole fraction')
ax3.set_xlabel('Stage Number')
ax3.legend()